In [1]:
import requests
import pytz
import datetime
import os
import csv

In [2]:
json_data = requests.get("https://aquametric.menon.pro/data/002/log.json").json()

In [3]:
date_format = '%Y-%m-%dT%H:%M:%S.%f%z'
my_timezone = pytz.timezone('US/Eastern')

thresh_date = datetime.datetime(2020, 4, 11, 16, 15, 0).replace(tzinfo=my_timezone)

def convert_date(date_str):
    return datetime.datetime.strptime(date_str, date_format).astimezone(my_timezone)

json_data = {convert_date(date): data for date, data in json_data.items() if convert_date(date) < thresh_date}

In [4]:
dates = sorted(list(json_data.keys()))
stages = [json_data[date]["data"]["stage"] for date in dates]
dates = [(date - dates[0]).total_seconds() / 60 for date in dates]
data = [[date, stage] for date, stage in zip(dates, stages)]

In [5]:
with open(os.path.expanduser('~/out.csv'), 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["Time (Minutes)", "Stage (mm)"])
    writer.writerows(data)